In [35]:
import pandas as pd
import numpy as np

#### Data Source
https://www.himalayandatabase.com/index.html

##### Guide
https://www.himalayandatabase.com/downloads/HD%20Online%20Guide.pdf

In [248]:
df_peaks = pd.read_csv("./Himalayan Database/RawData/peaksall.csv")
df_ex = pd.read_csv("./Himalayan Database/RawData/expeditions.csv")
df_mem = pd.read_csv("./Himalayan Database/RawData/members.csv")

C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [249]:
df_mem.columns

Index(['expid', 'membid', 'peakid', 'myear', 'mseason', 'fname', 'lname',
       'sex', 'age', 'birthdate', 'yob', 'calcage', 'citizen', 'status',
       'residence', 'occupation', 'leader', 'deputy', 'bconly', 'nottobc',
       'support', 'disabled', 'hired', 'sherpa', 'tibetan', 'msuccess',
       'mclaimed', 'mdisputed', 'msolo', 'mtraverse', 'mski', 'mparapente',
       'mspeed', 'mhighpt', 'mperhighpt', 'msmtdate1', 'msmtdate2',
       'msmtdate3', 'msmttime1', 'msmttime2', 'msmttime3', 'mroute1',
       'mroute2', 'mroute3', 'mascent1', 'mascent2', 'mascent3', 'mo2used',
       'mo2none', 'mo2climb', 'mo2descent', 'mo2sleep', 'mo2medical',
       'mo2note', 'death', 'deathdate', 'deathtime', 'deathtype', 'deathhgtm',
       'deathclass', 'msmtbid', 'msmtterm', 'hcn', 'mchksum', 'host', 'comrte',
       'stdrte', 'route1', 'route2', 'route3', 'route4', 'nation', 'leaders',
       'sponsor', 'termreason', 'totmembers', 'smtmembers', 'mdeaths',
       'tothired', 'nohired', 'smthire

### Countries

In [250]:
print(list(set(("/").join(df_mem.citizen.dropna().unique()).split("/"))))

['Argentina', 'Ukraine', 'Uzbekistan', 'S Korea', 'Norway', 'Zambia', 'Germany', 'Hong Kong', 'Ireland', 'Cyprus', 'Russia', 'Dominican Republic', 'W Germany', 'Turkey', 'Netherlands', 'Israel', 'Kenya', 'Egypt', 'New Zealand', 'Myanmar', 'Australia', 'UK', 'Slovenia', 'Mexico', 'USSR', 'Czechoslovakia', 'Yugoslavia', 'India?', 'Moldova', 'Jordan', 'S Africa', 'Azerbaijan', 'Portugal', 'Qatar', 'Taiwan', 'Tunisia', 'Finland', 'Switzerland', 'Sweden', 'Mongolia', 'Malaysia', 'Armenia', 'Mauritius', 'Iran', 'Georgia', 'Vietnam', 'France', 'Serbia', 'Denmark', 'Montenegro', 'Liechtenstein', 'Slovakia', 'Nepal', 'Luxembourg', 'Spain', 'Palestine', 'Kuwait', 'Thailand', 'Brazil', 'Bolivia', 'Panama', 'USA', 'Malta', 'N Korea', 'Japan', 'Czech Republic', 'Greece', 'San Marino', 'Bangladesh', 'Hungary', 'Ecuador', 'Costa Rica', 'Jamaica', 'Iceland', 'Croatia', 'Belgium', 'Turkmenistan', 'Estonia', 'Colombia', 'Guatemala', 'Lebanon', 'Bosnia-Herzegovina', 'Peru', 'Kyrgyz Republic', 'Poland', '

In [251]:
##df_mem.citizen.apply(lambda x : coco.convert(names=x, to='ISO2') if manual_map.get(x) == None  else manual_map.get(x))

### Get Members ready to join

In [252]:
df_mem['female']  = df_mem.sex.apply(lambda x : True if x=="F" else False)

In [253]:
## Clean age column
df_mem.calcage = df_mem.calcage.apply(lambda x : np.nan if x==0 else x)

In [254]:
df_mem.mspeed.value_counts()

False    65521
True        13
Name: mspeed, dtype: int64

In [255]:
gdf_mem = df_mem.groupby('peakid').agg({'expid' : 'count' ,'death' : 'sum', 'female' : 'sum', 'calcage' : 'mean', 'citizen' : 'unique','mo2used' : 'sum', "route1" : pd.Series.mode})

#### Grouby only the succeeding members

In [256]:
gdf_mem_success = df_mem[df_mem.msuccess == True].groupby('peakid').agg({'citizen' : 'unique', 'female' : 'sum','expid' : 'count' })

In [257]:
gdf_mem_success 

,citizen,female,expid
peakid,,,
ACHN,"[France, Nepal, Japan]",1.0,11
AMAD,"[W Germany, USA, Nepal, France, New Zealand, J...",322.0,3540
AMPG,"[UK, Nepal]",1.0,10
AMPH,[S Korea],1.0,3
AMPM,[Germany],1.0,5
...,...,...,...
URKM,"[Japan, Nepal, USA, Austria, France]",3.0,23
URMA,"[Taiwan, S Korea, Singapore, Australia, Nepal,...",11.0,52
YAKA,"[Japan, Nepal]",0.0,4


In [258]:
#gdf_mem.columns = ['_'.join(col).strip() for col in gdf_mem.columns.values]
#gdf_mem_success.columns = ['_'.join(col).strip() for col in gdf_mem_success.columns.values]

In [259]:
## Rename columns accordingly

In [260]:
gdf_mem.rename(columns={'expid' : 'count_mem_try', "female" :  "count_mem_try_female", "calcage" :"mean_mem_try_age", "citizen" : "country_mem_try", "route1" : "most_pop_route" }, inplace=True)
gdf_mem_success.rename(columns={'expid' : 'count_mem_success', "female" :  "count_mem_success_female", "calcage" :"mean_mem_success_age", "citizen" : "country_mem_success" }, inplace=True)

In [261]:
gdf_mem = pd.merge(gdf_mem,gdf_mem_success, on="peakid", how="left") 

In [262]:
gdf_mem.reset_index(inplace=True)

### Merge with Peak data

In [263]:
df = pd.merge(df_peaks,gdf_mem, on='peakid', how="left")

In [264]:
## WOrk with the ones with coords
df.dropna(subset=["lng"], inplace=True)

##### Adding ISO the the countries

In [265]:
import country_converter as coco

manual_map = {"USSR": 'USSR', "UK" : "GB", "UAE" : "AE", "W Germany" : "DE", "S Africa" : "ZA", "Yugoslavia" : "YUG" }


In [266]:
df.country_mem_success = df.country_mem_success.apply(lambda x : x if isinstance(x,float) else sum([i.split("/") for i in x ], []))

In [267]:
df.country_mem_success = df.country_mem_success.apply(lambda x : [ c if manual_map.get(c) == None else manual_map.get(c) for c in x ] if not isinstance(x,float) else x)

In [268]:
df.country_mem_success = df.country_mem_success.apply(lambda x : coco.convert(names=x, to='ISO2' , not_found=None) if not isinstance(x,float) else x)

YUG not found in ISO3
USSR not found in regex
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3


In [269]:
df.country_mem_success[~df.country_mem_success.isna()].apply(len)

0      151
1       45
2       54
3       11
4       46
      ... 
456      2
458      2
462      3
463      2
465      3
Name: country_mem_success, Length: 226, dtype: int64

### Line Chart data create

In [270]:
df_temp = df_mem.groupby(['peakid', 'myear']).agg({'membid' : 'count','msuccess' : 'sum','death' : 'sum'})

In [271]:
df_temp = df_temp.reset_index().sort_values(["peakid", 'myear'], ascending=[True,True]).groupby(['peakid' ]).agg(list).rename(columns={"myear" : "lineX", "membid" : "lineY_attemps", "msuccess" : "lineY_success", "death" : "lineY_death"} )

In [272]:
df = pd.merge(df,df_temp, on='peakid', how="left")

## Age data

In [273]:
def histo_transform(x) :
    return pd.cut(x, bins=[i *5 for i in range(21)   ],labels= ["{0} - {1}".format(i, i + 5) for i in range(0, 100, 5)]).value_counts().to_dict()

In [274]:
df_temp  = df_mem.groupby("peakid").agg({'calcage' : list}).rename(columns={"calcage" :"age_histogram" } )

In [275]:
df_temp.age_histogram = df_temp.age_histogram.apply(lambda x : histo_transform(x))

In [276]:
df = pd.merge(df,df_temp, on='peakid', how="left")

In [277]:
df.to_json("./Himalayan Database/GeoData/peaks2.json", orient='records')

In [278]:
df.to_csv("./Himalayan Database/GeoData/peaks2.csv")

In [279]:
df['value'] = df.apply(lambda x : [x.lat,x.lng, x.peakid], axis=1)
df['text'] = df.pkname + " - " + df.heightm.astype("str") + "m"

In [280]:
df[['value', 'text']].to_json("./assets/data/peaks_search.json" ,orient="records")

In [167]:
geojson = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [d["lon"], d["lat"]],
            },
        "properties" : d,
     } for d in data]
}

NameError: name 'data' is not defined

In [ ]:
df.to_dict(orient="records")